## Notebook to join two databases

The purpose of this notebook is to combine two databases (clinvar and longevitymap)

In [108]:
import polars as pl
from pathlib import *
import sqlite3
import pandas as pd
import pathlib

### Setting up paths

In [109]:
base = Path(".")
data = base / "data"
output = data / "output"
oakvar_module_path = base / ".." / "oakvar" / "modules"
oakvar_annotators = oakvar_module_path / "annotators"

longevitymap_path = oakvar_module_path/ "postaggregators" / "just_longevitymap" / "data" / "longevitymap.sqlite"
clinvar_path = oakvar_annotators / "clinvar" / "data" / "clinvar.sqlite"
ncbigene_path = oakvar_annotators / "ncbigene" / "data" / "ncbigene.sqlite"
print(longevitymap_path)

..\oakvar\modules\postaggregators\just_longevitymap\data\longevitymap.sqlite


In [110]:
longevitymap_conn = sqlite3.connect(str(longevitymap_path))

longevitymap_query = """SELECT rsid, chrom, pos, ref, alt, symbol
FROM snps, gene, variant WHERE
snps.rsid = variant.identifier AND variant.gene_id = gene.id;"""

df_pandas_longevitymap = pd.read_sql_query(longevitymap_query, longevitymap_conn)
longevitymap_df = pl.DataFrame(df_pandas_longevitymap)
longevitymap_conn.close()
longevitymap_df.head(3)

rsid,chrom,pos,ref,alt,symbol
str,str,i64,str,str,str
"""rs7412""","""chr19""",44908822,"""C""","""T""","""APOE"""
"""rs7412""","""chr19""",44908822,"""C""","""T""","""APOE"""
"""rs7412""","""chr19""",44908822,"""C""","""T""","""APOE"""


Cleaning data from duplicaitons:

In [111]:
longevitymap_df.describe

<bound method DataFrame.describe of shape: (2126, 6)
┌────────┬───────┬──────────┬─────┬─────┬────────┐
│ rsid   ┆ chrom ┆ pos      ┆ ref ┆ alt ┆ symbol │
│ ---    ┆ ---   ┆ ---      ┆ --- ┆ --- ┆ ---    │
│ str    ┆ str   ┆ i64      ┆ str ┆ str ┆ str    │
╞════════╪═══════╪══════════╪═════╪═════╪════════╡
│ rs7412 ┆ chr19 ┆ 44908822 ┆ C   ┆ T   ┆ APOE   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ rs7412 ┆ chr19 ┆ 44908822 ┆ C   ┆ T   ┆ APOE   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ rs7412 ┆ chr19 ┆ 44908822 ┆ C   ┆ T   ┆ APOE   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ rs7412 ┆ chr19 ┆ 44908822 ┆ C   ┆ T   ┆ APOE   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ ...    ┆ ...   ┆ ...      ┆ ... ┆ ... ┆ ...    │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ rs7412 ┆ chr19 ┆ 44908822 ┆ C   ┆ T   ┆ APOE   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ rs7412 ┆ chr19 ┆ 44908822 ┆ C   ┆ T   ┆ APOE   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼

In [112]:
longevitymap_df = longevitymap_df.unique()
longevitymap_df.describe

<bound method DataFrame.describe of shape: (857, 6)
┌───────────┬───────┬───────────┬─────┬─────────────────────────────────────┬──────────┐
│ rsid      ┆ chrom ┆ pos       ┆ ref ┆ alt                                 ┆ symbol   │
│ ---       ┆ ---   ┆ ---       ┆ --- ┆ ---                                 ┆ ---      │
│ str       ┆ str   ┆ i64       ┆ str ┆ str                                 ┆ str      │
╞═══════════╪═══════╪═══════════╪═════╪═════════════════════════════════════╪══════════╡
│ rs7412    ┆ chr19 ┆ 44908822  ┆ C   ┆ T                                   ┆ APOE     │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ rs1799752 ┆ chr17 ┆ 63488530  ┆ -   ┆ ATACAGTCACTTTTTTTTTTTTTTTGAGACGG... ┆ ACE      │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ rs1799889 ┆ chr7  ┆ 101126430 ┆ A   ┆ G                                   ┆ SERPINE1 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [113]:
ncbigene_conn = sqlite3.connect(str(ncbigene_path))
ncbigene_query = "SELECT gene_symbol, description FROM ncbigene;"
df_pandas_ncbigene = pd.read_sql_query(ncbigene_query, ncbigene_conn)
ncbigene_df = pl.DataFrame(df_pandas_ncbigene)
ncbigene_conn.close()
ncbigene_df = ncbigene_df.rename({"description":"gene_description"})
ncbigene_df = ncbigene_df.rename({"gene_symbol":"symbol"})
ncbigene_df.head(3)

symbol,gene_description
str,str
"""A1BG""","""The protein en..."
"""A2M""","""The protein en..."
"""NAT1""","""This gene is o..."


Joining ncbi db and longevity map df

In [115]:
lm_ncbi = longevitymap_df.join(ncbigene_df, how="left", on="symbol")
lm_ncbi.head(5)

rsid,chrom,pos,ref,alt,symbol,gene_description
str,str,i64,str,str,str,str
"""rs7412""","""chr19""",44908822,"""C""","""T""","""APOE""","""The protein en..."
"""rs1799752""","""chr17""",63488530,"""-""","""ATACAGTCACTTTT...","""ACE""","""This gene enco..."
"""rs1799889""","""chr7""",101126430,"""A""","""G""","""SERPINE1""","""This gene enco..."
"""rs1800795""","""chr7""",22727026,"""C""","""G""","""IL6""","""This gene enco..."
"""rs1800795""","""chr7""",22727026,"""C""","""T""","""IL6""","""This gene enco..."


Working on integrating clinvar db

In [122]:
rsid_dict = lm_ncbi.to_dicts()


Filter by curators comments and fix typos

In [123]:
clinvar_conn = sqlite3.connect(str(clinvar_path))
clinvar_cursor = clinvar_conn.cursor()

for i in rsid_dict:
    clinvar_query = """SELECT id, disease_refs, disease_names, rev_stat, sig,
    sig_conf FROM {chrom} WHERE pos = "{pos}"
    AND alt = "{alt}";""".format(chrom=i["chrom"], pos = i["pos"],
                                 alt=i["alt"])
    clinvar_cursor.execute(clinvar_query)
    row = clinvar_cursor.fetchone()
    if row is not None:
        for l in range(len(row)):
            i["clinvar_id"] = row[0]
            i["disease_refs"] = row[1]
            i["disease_names"] = row[2]
            i["rev_stat"] = row[3]
            i["sig"] = row[4]
            i["sig_conf"] = row[5]



clinvar_conn.close()
print(rsid_dict)

[{'rsid': 'rs7412', 'chrom': 'chr19', 'pos': 44908822, 'ref': 'C', 'alt': 'T', 'symbol': 'APOE', 'gene_description': 'The protein encoded by this gene is a major apoprotein of the chylomicron. It binds to a specific liver and peripheral cell receptor, and is essential for the normal catabolism of triglyceride-rich lipoprotein constituents. This gene maps to chromosome 19 in a cluster with the related apolipoprotein C1 and C2 genes. Mutations in this gene result in familial dysbetalipoproteinemia, or type III hyperlipoproteinemia (HLP III), in which increased plasma cholesterol and triglycerides are the consequence of impaired clearance of chylomicron and VLDL remnants. [provided by RefSeq, Jun 2016]', 'clinvar_id': 17848, 'disease_refs': 'Human Phenotype Ontology:HP:0003124,Human Phenotype Ontology:HP:0008154,Human Phenotype Ontology:HP:0008173,Human Phenotype Ontology:HP:0008359,MeSH:D006937,MedGen:C0020443,SNOMED CT:238076009|MONDO:MONDO:0007390,MedGen:C0750384,OMIM:122700|MONDO:MOND

In [124]:
final_db = pl.from_dicts(rsid_dict, schema_overrides={"rsid":pl.Int64, "chrom":pl.Utf8,
                                                      "pos":pl.Int64,"ref":pl.Utf8,"alt":pl.Utf8,
                                                       "symbol":pl.Utf8, "gene_description":pl.Utf8,
                                                       "clinvar_id":pl.Int64, "disease_refs": pl.Utf8, "disease_names":pl.Utf8,
                                                       "rev_stat":pl.Utf8, "sig": pl.Utf8, 
                                                       "sig_conf":pl.Utf8})
final_db.head()


TypeError: from_dicts() got an unexpected keyword argument 'schema_overrides'

In [120]:
new_sql_path = output / "ncbi_omim.sqlite3"
new_sql_conn= sqlite3.connect(new_sql_path)
neqwsql_curs = new_sql_conn.cursor()
final_db.to_pandas().to_sql("ncbi_clinvar", new_sql_conn, index=False)

857

In [121]:
new_sql_conn.close()